In [60]:
import random as rm
import numpy as np
import pandas as pd
#import os

##################################################################
# 1.) make an object instance
# 2.) a sequence is automatically generated
# 3.) call helper methods to get the sequence, transitions, or counts
##################################################################

class randGenerator():
    def __init__(self):
        self.filters = [0.25, 0.5, 0.75]
        self.counts = 0.1*np.ones(4)
        self.desired_len = 192
        self.sequence = np.zeros(self.desired_len, dtype=np.int)
        self.__generate()

    def __generate(self):
        self.filters = [0.25, 0.5, 0.75]
        self.sequence = np.zeros(self.desired_len, dtype=np.int)
        for i in range(self.desired_len):
            rand = rm.uniform(0,1)
            
            # or x = random.random()
            # or x = int.from_bytes(os.urandom(8), byteorder="big") / ((1 << 64) - 1)
    
            # if we are not at the very beginning of sequence
            # we can name the prev state
            if i != 0:
                prev = self.sequence[i - 1]       
    
            # otherwise we call it something impossible
            else:
                prev = -1
            
            next = self.__pickNext(prev,rand)
            self.__adjustFilters(next)
            self.sequence[i] = next
              
    def __pickNext(self, prev, rand):
        # select next state based on which interval contains the random number
        
        if rand < self.filters[0]:
            next = 0
        elif rand >= self.filters[0] and rand < self.filters[1]:
            next = 1
        elif rand >= self.filters[1] and rand < self.filters[2]:
            next = 2
        elif rand >= self.filters[2]:
            next = 3
        
        # if it happens to be equal to the previous state, try again
        
        if next == prev:
            return self.__pickNext(prev,rm.uniform(0,1))
        else:
            return next
        
    def __adjustFilters(self, next):
        self.__adjustCounts(next)
        N = np.sum(self.counts)
        self.filters[0] = self.counts[0]/N
        self.filters[1] = self.counts[1]/N + self.counts[0]/N
        self.filters[2] = self.counts[2]/N + self.counts[1]/N + self.counts[0]/N
   
    def __adjustCounts(self, next):
        for i in range(len(self.counts)):
            if i != next:
                self.counts[i] += np.sum(self.sequence)**2              # Can adjust weighting here
   
    ########################################################################
    # use these functions to access the sequence and elements of interest
    #########################################################################
    
    def getSequence(self):
        print(self.sequence)
    
    def getTransitions(self):
        transitions = np.zeros((4,4))
        
        for i in range(self.desired_len -1):
            transitions[self.sequence[i]][self.sequence[i+1]] +=1
    
        df = pd.DataFrame(transitions)
        print(df)
        
    def getCounts(self):
        unique, counts = np.unique(self.sequence, return_counts=True)
        print(dict(zip(unique, counts)))
        
    def getFilters(self):
        print(self.filters)

In [61]:
myobject = randGenerator()
myobject.getSequence()

[3 1 3 0 1 3 2 0 2 0 3 1 2 1 2 1 0 3 2 1 2 3 1 2 3 2 3 1 0 3 1 0 1 3 0 3 2
 0 3 2 3 1 2 1 3 1 2 0 3 1 3 0 2 1 0 3 0 2 3 2 3 0 1 0 2 1 2 3 0 1 0 1 3 2
 0 3 1 3 2 3 0 3 2 3 1 2 1 0 1 3 1 0 2 0 3 2 1 3 0 1 3 1 0 1 3 1 2 1 2 3 2
 1 3 2 1 3 2 3 1 0 1 0 1 2 0 1 0 1 0 3 0 3 1 3 2 3 1 3 0 3 0 1 2 1 0 2 1 2
 0 2 3 0 3 2 1 0 1 2 0 3 0 3 2 1 3 0 3 0 2 3 0 1 2 1 2 3 1 2 0 2 3 2 0 2 3
 2 0 2 3 0 3 1]


In [62]:
myobject.getTransitions()
myobject.getCounts()
myobject.getFilters()

      0     1     2     3
0   0.0  15.0  11.0  18.0
1  15.0   0.0  17.0  16.0
2  12.0  16.0   0.0  18.0
3  17.0  18.0  18.0   0.0
{0: 44, 1: 49, 2: 46, 3: 53}
[0.25032598666563993, 0.50732754698030591, 0.75436476715623058]


In [63]:
myobject2 = randGenerator()
myobject2.getSequence()

[2 3 2 1 3 1 3 2 1 0 2 0 3 0 2 1 0 1 3 2 3 0 2 1 0 1 2 1 2 1 3 2 1 2 3 2 1
 2 1 3 1 3 0 3 0 3 1 3 1 2 3 2 1 2 3 1 0 2 3 0 3 0 1 3 1 3 2 1 3 1 0 2 0 3
 1 2 1 0 2 0 1 3 2 1 0 2 3 0 1 2 0 3 1 0 1 0 1 0 2 3 2 0 3 2 3 2 1 2 3 0 2
 0 2 3 0 3 2 1 2 1 3 0 3 0 2 1 0 2 3 2 3 1 0 1 3 0 3 2 1 3 0 1 3 1 0 2 0 3
 1 3 0 3 0 1 3 0 1 0 2 3 2 3 2 1 0 3 2 1 0 2 3 0 3 1 2 0 2 1 3 0 3 0 3 1 3
 2 0 1 2 3 2 3]


In [64]:
myobject2.getTransitions()
myobject2.getCounts()
myobject2.getFilters()

      0     1     2     3
0   0.0  12.0  16.0  16.0
1  16.0   0.0  12.0  19.0
2   9.0  21.0   0.0  18.0
3  19.0  14.0  19.0   0.0
{0: 44, 1: 47, 2: 48, 3: 53}
[0.24927493188854341, 0.51119315282914768, 0.76597666989437574]
